In [16]:
import requests
import csv 

In [17]:
listoffiles=[]
def csvgetter(user,password,*otherusernames):
    #first create a session 's'
    s = requests.session()
    #this is the data that is sent to login to icheckmovies
    login_data={'login[username]': user,'login[password]': password,'submit': 'login',}
    #here we send the login data, using the session created before
    s.post("https://www.icheckmovies.com/login/", data=login_data) 
    #once again, whithin the session we get the csv link and call it 'r'
    usernames_list=list(otherusernames)
    usernames_list.append(user)
    for item in usernames_list:
        url='https://www.icheckmovies.com/movies/checked/?user='+item+'&export'
        r=s.get(url)
        checklist=r.text
        checklist_encode = checklist.encode('utf-8')
        filename=item+'_checklist'+'.csv'
        file=open(filename, "w")
        file.write(checklist_encode)
        file.close() 
        listoffiles.append(filename)
    numberofusers=str((len(listoffiles)))
    print('There are '+numberofusers+' users in the database: '+'\n'+'\n'.join(listoffiles))
    
def listmaker(username):
    filmlist=[]
    csvname=username+"_checklist.csv"
    csv_reader = csv.reader(open(csvname, "rb"), delimiter=';')
    for row in csv_reader:
        filmlist.append(row[0])
    return (filmlist)

def multiname(*usernames):
    for name in usernames:
        listmaker(name)
        listoffilmnames.append(filmlist)
        print(name)

now 'loginandgetcsv' will save a .csv in the parent directory called *username*_checklist.csv

In [48]:
username1=
username2=
username3=
password=x

SyntaxError: invalid syntax (<ipython-input-48-7190316a45b7>, line 1)

In [ ]:
csvgetter(username1,password,username2,username3)

In [50]:
listoffilmnames=[]

def multiname(*usernames):
    for name in usernames:
        listmaker(name)
        listoffilmnames.append(filmlist)
        print(name)

In [1]:
multiname(username1,username2,username3)

NameError: name 'multiname' is not defined